# U.S. Geological Survey Class GW3099

Advanced Modeling of Groundwater Flow (GW3099) <br>
Boise, Idaho <br>
September 16 - 20, 2024 <br>

# ![title](../../images/ClassLocation.jpg)

# Particle tracking example 3: forward tracking, transient flow

This notebook demonstrates particle tracking with a transient flow system, based on [Example 3](https://modflow6-examples.readthedocs.io/en/develop/_notebooks/ex-prt-mp7-p03.html) from the MODPATH 7 user guide. The flow system includes two wells, a drain, and a river.

The original problem runs PRT as a post-processor for a flow model. We demonstrate the use of a GWF-PRT exchange to combine the two models in a single simulation.

The original problem involves distinguishing particles terminating at each boundary. We extend the example to determine well pumping rates sufficient to keep the particles out of the drain and river.

Import dependencies.

In [ ]:
import pathlib as pl
import warnings

import flopy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

Enable inline plotting.

In [ ]:
%matplotlib inline

Ignore some warnings.

In [ ]:
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", DeprecationWarning)

Make sure MODPATH 7 is installed.

In [ ]:
flopy.utils.get_modflow(":python", subset="mp7")

Create a base workspace.

In [ ]:
example_name = "ex3"
base_ws = pl.Path("temp") / example_name
base_ws.mkdir(exist_ok=True, parents=True)

# Reproducing the example

We start by reproducing the results in the cited example, this time with a flow model and a tracking model occupying the same simulation.

First define a workspace and model names.

In [ ]:
gwf_name = f"{example_name}-gwf"
prt_name = f"{example_name}-prt"
demo_ws = base_ws / "demo"

Define model units.

In [ ]:
length_units = "feet"
time_units = "days"

Define flow model parameters.

In [ ]:
nper = 3  # Number of periods
nlay = 3  # Number of layers
nrow = 21  # Number of rows
ncol = 20  # Number of columns
delr = 500.0  # Column width ($ft$)
delc = 500.0  # Row width ($ft$)
top = 350.0  # Top of the model ($ft$)
botm = [220.0, 200.0, 0.0]  # Layer bottom elevations ($ft$)
kh = [50.0, 0.01, 200.0]  # Horizontal hydraulic conductivity ($ft/d$)
kv = [10.0, 0.01, 20.0]  # Vertical hydraulic conductivity ($ft/d$)
rchv = 0.005  # Recharge rate ($ft/d$)
riv_h = 320.0  # River stage ($ft$)
riv_z = 317.0  # River bottom ($ft$)
riv_c = 1.0e5  # River conductance ($ft^2/d$)
porosity = 0.1  # Soil porosity (unitless)

Define boundary conditions.

In [ ]:
# Define well data.
# Negative discharge indicates pumping, positive injection.
wells = [
    # layer, row, col, discharge
    (0, 10, 9, -75000),
    (2, 12, 4, -100000),
]

# Define the drain location.
drain = (0, 14, (9, 20))

# Define the river data.
riv_iface = 6
riv_iflowface = -1
rd = []
for i in range(nrow):
    rd.append(
        [(0, i, ncol - 1), riv_h, riv_c, riv_z, riv_iface, riv_iflowface]
    )

Calculate node numbers for the boundary conditions.

In [ ]:
nodes = {"well1": [], "well2": [], "drain": [], "river": []}
for idx, (k, i, j, _) in enumerate(wells):
    nodes[f"well{idx + 1}"].append(ncol * (nrow * k + i) + j)
for j in drain[2]:
    k, i = drain[:2]
    nodes["drain"].append([ncol * (nrow * k + i) + j])
for rivspec in rd:
    k, i, j = rivspec[0]
    node = ncol * (nrow * k + i) + j
    nodes["river"].append(node)

Configure a zone mapping.

In [ ]:
def get_izone():
    izone = []

    # zone 1 is the default (non-terminating)
    def ones():
        return np.ones((nrow, ncol), dtype=np.int32)

    # layer 1
    l1 = ones()
    l1[wells[0][1:3]] = 2  # well
    l1[drain[1], drain[2][0] : drain[2][1]] = 4  # drain
    l1[:, ncol - 1] = 5  # river
    izone.append(l1)

    # layer 2
    izone.append(1)

    # layer 3
    l3 = ones()
    l3[wells[1][1:3]] = 3  # well
    izone.append(l3)

    return izone


izone = get_izone()

Construct a simulation.

In [ ]:
sim = flopy.mf6.MFSimulation(
    sim_name=example_name, exe_name="mf6", version="mf6", sim_ws=demo_ws
)

Create the time discretization.

In [ ]:
tdis_pd = [
    # perlen, nstp, tsmult
    (100000, 1, 1),
    (36500, 10, 1.5),
    (100000, 1, 1),
]
tdis = flopy.mf6.modflow.mftdis.ModflowTdis(
    sim,
    pname="tdis",
    time_units=time_units,
    nper=len(tdis_pd),
    perioddata=tdis_pd,
)

Create the groundwater flow model.

In [ ]:
model_nam_file = f"{gwf_name}.nam"
gwf = flopy.mf6.ModflowGwf(
    sim, modelname=gwf_name, model_nam_file=model_nam_file, save_flows=True
)

Create the iterative model solver package for the flow model.

In [ ]:
ims = flopy.mf6.modflow.mfims.ModflowIms(
    sim,
    pname="ims",
    complexity="SIMPLE",
)
sim.register_solution_package(ims, [gwf.name])

Create the grid discretization.

In [ ]:
dis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(
    gwf,
    pname="dis",
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    length_units="FEET",
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)

Create the initial conditions package.

In [ ]:
ic = flopy.mf6.modflow.mfgwfic.ModflowGwfic(gwf, pname="ic", strt=320)

Create the node property flow package.

In [ ]:
npf = flopy.mf6.modflow.mfgwfnpf.ModflowGwfnpf(
    gwf,
    pname="npf",
    icelltype=[1, 0, 0],  # top layer convertible
    k=kh,
    k33=kv,
    save_flows=True,
    save_specific_discharge=True,
    save_saturation=True,
)

Create the recharge package.

In [ ]:
rch = flopy.mf6.modflow.mfgwfrcha.ModflowGwfrcha(gwf, recharge=rchv)

Create the storage package.

In [ ]:
sto = flopy.mf6.modflow.mfgwfsto.ModflowGwfsto(
    gwf,
    save_flows=True,
    iconvert=1,
    ss=0.0001,
    sy=0.1,
    steady_state={0: True, 2: True},
    transient={1: True},
)

Create the well package.

In [ ]:
def no_flow(w):
    return w[0], w[1], w[2], 0


nf_wells = [no_flow(w) for w in wells]
wel = flopy.mf6.modflow.mfgwfwel.ModflowGwfwel(
    gwf,
    maxbound=2,
    stress_period_data={0: nf_wells, 1: wells, 2: wells},
)

Create the river package.

In [ ]:
riv = flopy.mf6.modflow.mfgwfriv.ModflowGwfriv(
    gwf, auxiliary=["iface", "iflowface"], stress_period_data={0: rd}
)

Create the drain package.

In [ ]:
drn_iface = 6
drn_iflowface = -1
dd = [
    [
        drain[0],
        drain[1],
        i + drain[2][0],
        322.5,
        100000.0,
        drn_iface,
        drn_iflowface,
    ]
    for i in range(drain[2][1] - drain[2][0])
]
drn = flopy.mf6.modflow.mfgwfdrn.ModflowGwfdrn(
    gwf,
    auxiliary=["iface", "iflowface"],
    maxbound=11,
    stress_period_data={0: dd},
)

Create the flow model output control package.

In [ ]:
headfile = f"{gwf_name}.hds"
budgetfile = f"{gwf_name}.cbb"
oc = flopy.mf6.modflow.mfgwfoc.ModflowGwfoc(
    gwf,
    pname="oc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    head_filerecord=[headfile],
    budget_filerecord=[budgetfile],
)

Create the particle tracking model.

In [ ]:
prt = flopy.mf6.ModflowPrt(
    sim, modelname=prt_name, model_nam_file="{}.nam".format(prt_name)
)

Create the particle tracking model's discretization package.

In [ ]:
prtdis = flopy.mf6.modflow.mfgwfdis.ModflowGwfdis(
    prt,
    pname="dis",
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    length_units="FEET",
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)

Create the particle tracking model input package. This includes global settings like porosity and the zone mapping.

In [ ]:
mip = flopy.mf6.ModflowPrtmip(prt, pname="mip", porosity=porosity, izone=izone)

Create a particle release package. For convenience, we will define a particle release template for MODPATH 7 and convert it to PRT format.

We use the `localz` option to position release points vertically with respect to the water table.

In [ ]:
# Define a MODPATH 7 release configuration.
rel_minl = rel_maxl = 0
rel_minr = 2
rel_maxr = 3
rel_minc = 2
rel_maxc = 3
celldata = flopy.modpath.CellDataType(
    drape=0,
    rowcelldivisions=5,
    columncelldivisions=5,
    layercelldivisions=1,
)
lrcregions = [[rel_minl, rel_minr, rel_minc, rel_maxl, rel_maxr, rel_maxc]]
lrcpd = flopy.modpath.LRCParticleData(
    subdivisiondata=[celldata],
    lrcregions=[lrcregions],
)

Exercise: define an equivalent configuration without using any MP7 utilities.

Define release times. The first release occurs 90,000 days into the simulation, with releases continuing every 20 days, for 200 days.

In [ ]:
release_times = list(range(90000, 90200, 20))

We can now create the particle release point package.

In [ ]:
release_points = list(lrcpd.to_prp(prt.modelgrid, localz=True))
prp = flopy.mf6.ModflowPrtprp(
    prt,
    pname="prp1",
    filename="{}_1.prp".format(prt_name),
    nreleasepts=len(release_points),
    packagedata=release_points,
    nreleasetimes=len(release_times),
    releasetimes=[(t,) for t in release_times],
    # local z coordinates specified, compute global release
    # coord from cell top if saturated or water table if not
    local_z=True,
    exit_solve_tolerance=1e-5,
    extend_tracking=True,
)

Create the particle tracking model output control package.

In [ ]:
budgetfile_prt = f"{prt_name}.cbb"
trackfile_prt = f"{prt_name}.trk"
trackhdrfile_prt = f"{prt_name}.trk.hdr"
trackcsvfile_prt = f"{prt_name}.trk.csv"
track_times = list(range(90000, 150001, 2000))
oc = flopy.mf6.ModflowPrtoc(
    prt,
    pname="oc",
    budget_filerecord=[budgetfile_prt],
    trackcsv_filerecord=[trackcsvfile_prt],
    track_release=True,
    track_terminate=True,
    track_usertime=True,
    ntracktimes=len(track_times),
    tracktimes=[(t,) for t in track_times],
    saverecord=[("BUDGET", "ALL")],
)

Create the PRT flow model interface.

In [ ]:
exg = flopy.mf6.ModflowGwfprt(
    sim,
    exgtype="GWF6-PRT6",
    exgmnamea=gwf_name,
    exgmnameb=prt_name,
    filename=f"{gwf_name}.gwfprt",
)

Create an explicit model solution for the particle tracking model.

In [ ]:
ems = flopy.mf6.ModflowEms(
    sim,
    pname="ems",
    filename=f"{prt_name}.ems",
)
sim.register_solution_package(ems, [prt.name])

Write and run the simulation.

In [ ]:
sim.write_simulation()
sim.run_simulation(silent=False)

Retrieve head results.

In [ ]:
gwf = sim.get_model(gwf_name)
head = gwf.output.head().get_data()

Retrieve pathlines.

In [ ]:
prt = sim.get_model(prt_name)
pathlines = pd.read_csv(demo_ws / trackcsvfile_prt)
pathlines

Plot results.

In [ ]:
from matplotlib.patches import Patch


def plot_heads(ax, gwf, heads):
    mm = flopy.plot.PlotMapView(gwf, ax=ax, layer=2)
    mm.plot_grid(alpha=0.25)
    mm.plot_bc("WEL", plotAll=True, color="red")
    mm.plot_bc("RIV", plotAll=True, color="blue")
    mm.plot_bc("DRN", plotAll=True, color="green")
    pc = mm.plot_array(heads, edgecolor="black", alpha=0.25)
    cb = plt.colorbar(pc, shrink=0.25, pad=0.1)
    cb.ax.set_xlabel(r"Head ($ft$)")
    return mm


with flopy.plot.styles.USGSPlot():
    fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
    ax.set_aspect("equal")
    ax.legend(
        handles=[
            Patch(color="red", label="Well"),
            Patch(color="blue", label="River"),
            Patch(color="green", label="Drain"),
        ],
    )
    fig.tight_layout()
    flopy.plot.styles.heading(ax, heading="Head, layer 3")
    mm = plot_heads(ax, gwf, head[2, :, :])
    mm.plot_pathline(
        pathlines, layer="all", colors="black", alpha=0.5, linewidth=0.1
    )

## Exercise

Imagine the plume in this example is from a contaminant source. The demonstrated pumping scheme diverts some of it from the river, but not all.

1. Assume we can pump only the well in the top layer, and only at the same rate. Could we have diverted the plume, had we known of it and began to pump immediately?  If so, at what time does it become impossible to divert?

2. Assume we just learned of the plume, and we estimate it to be 10,000 days old (27 years and several months). This matches the original time discretization, where pumping begins 10,000 days after particle release, 100,000 days into the simulation. We'd like to keep the plume out of the drain/stream with a minimum of pumping from the two wells. How much should we pump from each well?


<details>
  <summary>Hints</summary>
  
This will involve counting how many particles terminate in each of the boundaries (wells, drain and river). Recall how to select terminating points in example notebook 1, then use the zone map.

To answer the first question, we can start pumping the first well at particle release time. If the plume is diverted, we can rerun the model several times, advancing the time at which we start pumping by a small increment, until the plume is no longer captured.

To answer the second question, we can modify pumping rates until the particles are all captured by the wells.

To get a better view of where particles are going, it might be helpful to plot the model in 3D with PyVista.
</details>